In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl
from split_dataset import SplitDataset

import colorspacious
import matplotlib.pyplot as plt
import json
from pathlib import Path
import tifffile as tiff

In [ ]:
master = Path(r"Z:\Hagar\E0040\v31\pre ablation 2p")
fish_list = list(master.glob("*_f*"))
path = fish_list[0]
print(path)

In [ ]:
suite2p_data = fl.load(path / "data_from_suite2p_cells.h5")
anatomy = suite2p_data['anatomy_stack']
traces = fl.load(path / "filtered_traces.h5", "/detr")
coords = suite2p_data['coords']

In [ ]:
ref_path = Path(r"Z:\Hagar\mpin_zfish_1um_v1.1\H2BGCaMP.tiff")
ref = tiff.imread(ref_path)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8,3), gridspec_kw={'width_ratios': [1,2]})

In [ ]:
axs[0].set_ylim(800, 0)

In [ ]:
axs[0].imshow(np.sum(ref, axis=1), cmap="gray_r", origin='upper')
axs[0].axis('off')

In [ ]:
axs[1].text(0, 100, "Heading direction neurons", fontsize=9)
axs[1].text(0, 60, "Motor neurons", fontsize=9)
axs[1].text(0, 23, "Sensory neurons", fontsize=9)

In [ ]:
file_name = "aim2 with confocal ref legend.pdf"
fig.savefig(path / file_name, dpi=300)

In [ ]:
axs[0].imshow(np.sum(anatomy, axis=0), cmap="gray_r", origin='lower')
axs[0].axis('off')

In [ ]:
file_name = "aim2 with confocal anatomy.pdf"
fig.savefig(path / file_name, dpi=300)

In [ ]:
sens_regs = fl.load(path / "sensory_regressors.h5", "/regressors")
left = np.asarray(sens_regs.iloc[:, 0])
right = np.asarray(sens_regs.iloc[:, 4])
num_traces = np.shape(traces)[1]

reg_corr_left = np.zeros((num_traces))
reg_corr_right = np.zeros((num_traces))

for i in range(num_traces):
    reg_corr_left[i] = np.corrcoef(left, traces[:, i])[0,1]
    reg_corr_right[i] = np.corrcoef(right, traces[:, i])[0,1]


mp_ind_l = np.argsort(np.abs(reg_corr_left))
mp_ind_r = np.argsort(np.abs(reg_corr_right))

In [ ]:
reg_corr_left[mp_ind_l]

In [ ]:
l_traces = traces[:, mp_ind_l[-5:]]
r_traces = traces[:, mp_ind_r[-5:]]

In [ ]:
t_sensory =  np.arange(0, np.shape(traces)[0]) / 2

In [ ]:
for i in range(2):
    axs[1].plot(t_sensory, l_traces[:,i] + i*5, c='skyblue')
    
    axs[1].plot(t_sensory, r_traces[:,i] + i*5 + 10, c='skyblue')

In [ ]:
axs[1].set_xlim(0, 1800)

In [ ]:
axs[1].axis('off')

In [ ]:
hdns_path = Path(r"\\funes2\legacy\experiments\E0071_lotr\full_ring\210511_f0\210511_f0b_natmov")

In [ ]:
hdns_traces = fl.load(hdns_path / "filtered_traces.h5", "/detr")
hdns_idx = fl.load(hdns_path / "selected.h5")
hdns = hdns_traces[:,hdns_idx]

In [ ]:
t_hdn = np.arange(0, np.shape(hdns_traces)[0]) / 5

In [ ]:
for i in range(2):
    axs[1].plot(t_hdn, hdns[:,i] + i*5 + 80, c='orchid')
    
    axs[1].plot(t_hdn, hdns[:,i+20] + i*5 + 90, c='orchid')

In [ ]:
mot_regs = fl.load(hdns_path / "motor_regressors.h5")

In [ ]:
reg_mot_left = np.asarray(mot_regs['left_bias'])
reg_mot_right = np.asarray(mot_regs['right_bias'])

In [ ]:
mp_ind_lmot = np.argsort(np.abs(reg_mot_left))
mp_ind_rmot = np.argsort(np.abs(reg_mot_right))

l_mot_traces = hdns_traces[:, mp_ind_lmot[-5:]]
r_mot_traces = hdns_traces[:, mp_ind_rmot[-5:]]

In [ ]:
for i in range(2):
    axs[1].plot(t_hdn, l_mot_traces[:,i] + i*5 + 40, c='coral')
    
    axs[1].plot(t_hdn, r_mot_traces[:,i] + i*5 + 50, c='coral')